In this notebook I'm going to experiment with using elo to predict match outcomes.

In [1]:
import pandas as pd
import numpy as np

In [2]:
deuce_atp = pd.read_csv('Data/atp_matches_deuce.csv')

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def data_cleaning(df, 
                  tourneys_to_include = ['Grand Slams', 'Masters', '250 or 500', 'Tour Finals', 'Davis Cup'], 
                  start_year=2000 ):
    
    #Renaming columns
    new_cols = [
    'Unnamed: 0', 'tourney_id', 'tourney_name', 'surface', 'draw_size',
       'tourney_level', 'match_num', 'winner_id', 'winner_seed',
       'winner_entry', 'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc',
       'winner_age', 'winner_rank', 'winner_rank_points', 'loser_id',
       'loser_seed', 'loser_entry', 'loser_name', 'loser_hand', 'loser_ht',
       'loser_ioc', 'loser_age', 'loser_rank', 'loser_rank_points', 'score',
       'best_of', 'round', 'minutes', 'winner_ace', 'winner_df', 'winner_svpt', 'winner_1stIn',
       'winner_1stWon', 'winner_2ndWon', 'winner_SvGms', 'winner_bpSaved', 'winner_bpFaced', 'loser_ace',
       'loser_df', 'loser_svpt', 'loser_1stIn', 'loser_1stWon', 'loser_2ndWon', 'loser_SvGms',
       'loser_bpSaved', 'loser_bpFaced', 'W1', 'W2', 'W3', 'W4', 'W5', 'L1', 'L2',
       'L3', 'L4', 'L5', 'retirement', 'WTB1', 'LTB1', 'WTB2', 'LTB2', 'WTB3',
       'LTB3', 'WTB4', 'LTB4', 'WTB5', 'LTB5', 'tourney_start_date', 'year',
       'match_id'
    ]
    
    df.columns = new_cols
    
    #You can change what matches to include. I've chosen to exclude Futures matches and the Challenger tour
    # tourney_levels = 'Grand Slams', '250 or 500', 'Davis Cup', 'Masters', 'Challenger', 'Tour Finals', 'Futures'
    df = df[(df['tourney_level'].isin(tourneys_to_include)) &\
            (df['year'] >= start_year) & (df['surface'] == 'Hard')&\
            (~df['round'].isin(['Q1', 'Q2', 'Q3', 'Q4']))
           ]

    #Converting dates to datetime
    df.loc[:,'tourney_start_date'] = pd.to_datetime(df['tourney_start_date'])
    df.loc[:,'year'] = pd.to_datetime(df['year'])
    
    #Parsing scores
    scores = df.loc[:,'score'].str.split(' ')
    scores = scores.fillna(0)     
    loser_total_games = []
    winner_total_games = []
    
    for index, value in scores.items():
        loser_game_score = 0
        winner_game_score = 0
        try:
            if value == 0 or value == ['W/O']:            
                loser_total_games.append(loser_game_score)
                winner_total_games.append(winner_game_score)

            else:
                loser_game_score = 0
                winner_game_score = 0

                for set_ in value:                
                    try:
                        text = re.match(r"(\d)\-(\d)",set_ )
                        loser_game_score += int(text.group(2))
                        winner_game_score += int(text.group(1))
                    except:
                        pass
                loser_total_games.append(loser_game_score)
                winner_total_games.append(winner_game_score)
        except:
            print(index, value)

    df.loc[:,'winner_total_games'] = winner_total_games
    df.loc[:,'loser_total_games'] = loser_total_games
    df.loc[:,'total_games'] = df['winner_total_games'] + df['loser_total_games']
    df.loc[:,'loser_RtGms'] = df['winner_SvGms']
    df.loc[:,'winner_RtGms'] = df['loser_SvGms']
    df.loc[:,'loser_bp'] = df['winner_bpFaced']
    df.loc[:,'winner_bp'] = df['loser_bpFaced']


    df.loc[:,'loser_bpWon'] = df['winner_bpFaced'] - df['winner_bpSaved'] 
    df.loc[:,'winner_bpWon'] = df['loser_bpFaced'] - df['loser_bpSaved'] 
    
    #Imputing returns data so we can construct features
    df.loc[:,'winner_2ndIn'] = df['winner_svpt'] - df['winner_1stIn'] - df['winner_df']
    df.loc[:,'loser_2ndIn'] = df['loser_svpt'] - df['loser_1stIn'] - df['loser_df']
    df.loc[:,'loser_rtpt'] = df['winner_svpt']
    df.loc[:,'winner_rtpt'] = df['loser_svpt']
    df.loc[:,'winner_rtptWon'] = df['loser_svpt'] -  df['loser_1stWon'] - df['loser_2ndWon']
    df.loc[:,'loser_rtptWon'] = df['winner_svpt'] -  df['winner_1stWon'] - df['winner_2ndWon']
    df.loc[:,'winner_svptWon'] = df['winner_1stWon'] + df['winner_2ndWon']
    df.loc[:,'loser_svptWon'] = df['loser_1stWon'] + df['loser_2ndWon']
    df.loc[:,'winner_total_points'] = df['winner_svptWon'] + df['winner_rtptWon']
    df.loc[:,'loser_total_points'] = df['loser_svptWon'] + df['loser_rtptWon']
    df.loc[:,'total_points'] = df['winner_total_points'] + df['loser_total_points']
    
    #Dropping columns
    cols_to_drop =[
        'draw_size',
        'winner_seed',
        'winner_entry',
        'loser_seed',
        'loser_entry',
        'score',
        'W1', 'W2', 'W3', 'W4', 'W5', 'L1', 'L2',
        'L3', 'L4', 'L5', 'WTB1', 'LTB1', 'WTB2', 'LTB2', 'WTB3',
        'LTB3', 'WTB4', 'LTB4', 'WTB5', 'LTB5'
        ]
    
    df.drop(cols_to_drop, axis=1, inplace=True)
    
    #Filling nans values
    df.loc[:,'loser_rank'] = df['loser_rank'].fillna(500)
    df.loc[:,'winner_rank'] = df['winner_rank'].fillna(500)
    df = df.fillna(df.mean())
    
    return(df)

def convert_long(df):
    
    #Separating features into winner and loser so we can create rolling averages for each major tournament
    winner_cols = [col for col in df.columns if col.startswith('w')]
    loser_cols = [col for col in df.columns if col.startswith('l')]
    common_cols = [
        'tourney_id', 'tourney_name', 'surface', 'tourney_level',
       'match_num','best_of', 'round',
       'minutes','retirement', 'tourney_start_date', 'year', 'match_id',
        'total_points', 'total_games'
    ]
    
    #Will also add opponent's rank
    df_winner = df[winner_cols + common_cols + ['loser_rank']]
    df_loser = df[loser_cols + common_cols + ['winner_rank']]
    
    df_winner['won'] = 1
    df_loser['won'] = 0
    
    #Renaming columns
    df_winner.columns = [col.replace('winner','player').replace('loser', 'opponent') for col in df_winner.columns]
    df_loser.columns = df_winner.columns
    
    df_long = df_winner.append(df_loser, ignore_index=True)
    
    return(df_long)

def get_new_features(df):
    
    #Creating new features we can play around with, note that not all features may be used
    df.loc[:,'player_serve_win_ratio'] = (df['player_1stWon'] + df['player_2ndWon'])/\
    (df['player_1stIn'] + df['player_2ndIn'] + df['player_df'] )
    
    df.loc[:,'player_return_win_ratio'] = df['player_rtptWon']/df['player_rtpt']
    
    df.loc[:,'player_bp_per_game'] = df['player_bp']/df['player_RtGms']
    
    df.loc[:,'player_bp_conversion_ratio'] = df['player_bpWon']/df['player_bp']
    
    #Setting nans to zero for breakpoint conversion ratio
    df.loc[:,'player_bp_conversion_ratio'].fillna(0, inplace=True)
    
    df.loc[:,'player_game_win_ratio'] = df['player_total_games']/df['total_games']
    
    df.loc[:,'player_point_win_ratio'] = df['player_total_points']/df['total_points']
    
    #df['player_set_Win_Ratio'] = df['Player_Sets_Won']/df['Total_Sets']
    
    df.loc[:,'player_clutch_factor'] = df['player_game_win_ratio'] - df['player_point_win_ratio']
    
    df.loc[:,'player_log_rank'] = np.log(df['player_rank'])
    
    df.loc[:,'player_log_rank_points'] = np.log(df['player_rank_points'])
    
    df.loc[:,'player_win_weight'] = df['won'] * np.exp(-df['opponent_rank']/100)

    #Let's try weighting some of the features by the opponent's rank
    
    #df['Player_Set_Win_Ratio_Weighted'] = df['Player_Set_Win_Ratio']*np.exp((df['Player_Rank']-df['Opponent_Rank'])/500)
    df.loc[:,'player_game_win_ratio_weighted'] = df['player_game_win_ratio']*np.exp((df['player_rank']-df['opponent_rank'])/500)
    df.loc[:,'player_point_win_ratio_weighted'] = df['player_point_win_ratio']*np.exp((df['player_rank']-df['opponent_rank'])/500)
    
    return(df)

def get_rolling_features(df, date_df, rolling_cols, last_cols, window):
    
    #This code is basically copied straight from Qile Tan's notebook
    
    df = df.sort_values(['player_name', 'tourney_name', 'tourney_start_date'], ascending=True)
    
    for index, tournament_date in enumerate(date_df.tourney_start_date):
        print(index, tournament_date)
        
        #Subsetting to tournaments at most 1 year before tournament date to reduce computation time
        df_temp = df.loc[(df['tourney_start_date']< tournament_date) & (df['tourney_start_date'] > tournament_date - datetime.timedelta(days=365))]

        #Only taking the most recent value for the feature, if specified in last_cols
        if last_cols != None:
            df_temp_last = df_temp.groupby('player_name')[last_cols].last().reset_index()

        #Taking a rolling average of the x (window_length) most recent matches before specified tournament date,
        #for features specified in rolling_cols
        df_temp = df_temp.groupby('player_name')[rolling_cols].rolling(window,1).mean().reset_index()

        #Only taking the most recent rolling average
        df_temp = df_temp.groupby('player_name').tail(1)

        df_temp = df_temp.merge(df_temp_last, on = 'player_name', how='left')

        #Adding a column telling us what tournament the rolling average is for
        if index == 0:
            df_result = df_temp
            df_result['tournament_date_index'] = tournament_date

        else:
            df_temp['tournament_date_index'] = tournament_date
            df_result = df_result.append(df_temp)
        
    
    df_result.drop('level_1', axis=1, inplace=True)
    
    return(df_result)

def merge_data(df, df_rolling_atp):
    
    df_atp = df.copy()
    #Subsetting match data to Grand Slams and Masters
    df_atp = df_atp.loc[df_atp['tourney_level'].isin(['Grand Slams', 'Masters'])]

    #Removing unnecessary columns from match data
    cols_to_keep = ['winner_name','loser_name','tourney_name','tourney_start_date', 'tourney_level', 'round']

    df_atp = df_atp[cols_to_keep]
    df1 = df_atp.copy()
    df1.columns = ['player_1','player_2','tourney_name','tourney_start_date', 'tourney_level', 'round']
    df1['player_1_win'] = 1

    df2 = df_atp.copy()
    df2.columns = ['player_2','player_1','tourney_name','tourney_start_date', 'tourney_level', 'round']
    df2['player_1_win'] = 0

    df_atp = pd.concat([df1, df2], sort=False)
    df_atp.reset_index(drop=True, inplace=True)
    

    #Joining rolling features for p1 with match data
    df_atp = df_atp.merge(df_rolling_atp, how='left',
                         left_on = ['player_1', 'tourney_start_date'],
                         right_on = ['player_name', 'tournament_date_index'],
                         validate = 'm:1')


    df_atp = df_atp.merge(df_rolling_atp, how='left',
                         left_on = ['player_2', 'tourney_start_date'],
                         right_on = ['player_name', 'tournament_date_index'],
                         validate = 'm:1',
                         suffixes=('_p1', '_p2'))
    
    return(df_atp)

def get_player_difference(df, diff_cols = None):
    
    p1_cols = [i + '_p1' for i in diff_cols] # column names for player 1 stats
    p2_cols = [i + '_p2' for i in diff_cols] # column names for player 2 stats


    # Filling missing values
    df['player_rank_p1'] = df['player_rank_p1'].fillna(500)
    df['player_log_rank_p1'] = df['player_log_rank_p1'].fillna(np.log(500))
    df[p1_cols] = df[p1_cols].fillna(-1)
    
    df['player_rank_p2'] = df['player_rank_p2'].fillna(500)
    df['player_log_rank_p2'] = df['player_log_rank_p2'].fillna(np.log(500))
    df[p2_cols] = df[p2_cols].fillna(-1)

    
    new_column_name = [i + '_diff' for i in diff_cols]

    # Take the difference
    df_p1 = df[p1_cols]
    df_p2 = df[p2_cols]
    
    df_p1.columns=new_column_name
    df_p2.columns=new_column_name
    
    df_diff = df_p1 - df_p2
    df_diff.columns = new_column_name
    
    #Dropping spare columns
    df.drop(p1_cols + p2_cols, axis=1, inplace=True)
    
    # Concat the df_diff and raw_df
    df = pd.concat([df, df_diff], axis=1)
    
    return(df)



In [4]:
deuce_atp = data_cleaning(deuce_atp)

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docume

In [5]:
deuce_atp = pd.read_csv('Data/atp_matches_deuce.csv')
all_names = list(set(deuce_atp.loser_name.unique()) | set(deuce_atp.winner_name.unique()))

In [6]:
elo_dict = {player:[1500] for player in all_names}

i = 0

previous_winner_elos = []
previous_loser_elos = []

for index, row in deuce_atp.iterrows():
    
    winner = row.winner_name
    loser = row.loser_name
    
    winner_elos = elo_dict[winner]
    loser_elos = elo_dict[loser]
    
    winner_matches = len(winner_elos)
    loser_matches = len(loser_elos)
    
    winner_old_elo = winner_elos[-1]
    loser_old_elo = loser_elos[-1]
    
    previous_winner_elos.append(winner_old_elo)
    previous_loser_elos.append(loser_old_elo)
    
    pr_p1_win_elo = 1/(1+10**((loser_old_elo - winner_old_elo)/400))
    pr_p2_win_elo = 1/(1+10**((winner_old_elo - loser_old_elo)/400))
    
    winner_K = 250/((winner_matches+5)**0.4)
    loser_K = 250/((loser_matches+5)**0.4)
    
    winner_new_elo = winner_old_elo + winner_K*(1-pr_p1_win_elo)
    loser_new_elo = loser_old_elo + loser_K*(0-pr_p2_win_elo)
    
    winner_elos.append(winner_new_elo)
    loser_elos.append(loser_new_elo)
    
    elo_dict[winner] = winner_elos
    elo_dict[loser] = loser_elos

In [7]:
deuce_atp.loc[:,'winner_old_elo'] = previous_winner_elos
deuce_atp.loc[:,'loser_old_elo'] = previous_loser_elos

In [9]:
deuce_atp.to_csv('Data/atp_matches_elo.csv', index=False)